<a href="https://colab.research.google.com/github/Jaacoo/CNN_ImageClassification/blob/master/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification

In this project we are going to use Tensorflow Keras and a Convolutional Neural Network (CNN).

Before we start, we need to import several libraries and modules.

In [0]:
#numpy is a library for scientific calculation. It consists of multidimensional array objects known as ndarray objects
import numpy as np 
# OpenCV (here imported as cv2) is a library designed to solve computer vision problems
import cv2
# This module is for using operating system dependent functionality
import os

#importing TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import utils
from keras.layers.normalization import BatchNormalization

from sklearn.utils import shuffle

Using TensorFlow backend.


In this step we are going to prepare our data. We are extracting the images and labeling them.

In [0]:
# In 'data_path' all our data / images are stored 
def extract_imgs(data_path, count):
  image_list = []
  label_list = []
  lab = 0
    
  #iterating through the folder structure
  for label in os.listdir(data_path):
    if label == 'buildings': 
      lab = 0
    elif label == 'forest':
      lab = 1
    elif label == 'glacier':
      lab = 2
    elif label == 'mountain':
      lab = 3
    elif label == 'sea':
      lab = 4
    elif label == 'street':
      lab = 5
    # iterate through the above folder structure and take an image from current 
    # folder and resize it (images have different sizes in the data set)
    # at the end append the current image and the corresponding label
    i = 0
    for image_file in os.listdir(data_path+label): # Extracting the file name of the image from Class Label folder
      image = cv2.imread(data_path+label+r'/'+image_file) # Reading the image (OpenCV)
      image = cv2.resize(image,(256,256)) # Resize the images to 256x256
      image_list.append(image)
      label_list.append(lab)
      i=i+1
      # If only 'count' images are required
      if i == count:
        i = 0
        break
  return (np.array(image_list), np.array(label_list))

#shuffling ordered data
def shuffle_data(images, labels):
  return shuffle(images, labels, random_state=1)

Extracting data from folders and exploring the data set.

In [0]:
X_train_raw, y_train_raw = extract_imgs('/content/drive/My Drive/ImageClassification/seg_train/',2100)
X_train, y_train = shuffle_data(X_train_raw, y_train_raw)

In this step we are going to build the model. It is going to be a CNN with several convolution and max pooling layers.
With Batch Normalization we normalize the activations of the previous layer at each batch.

After Convolution and Max Pool layers we are going to add fully connected layers. In order to avoid overfitting we are going to use "Drop out". 

In [0]:
model = models.Sequential()

#1st Convolutional Layer
model.add(layers.Conv2D(input_shape=(256,256,3), filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'))
#Max Pooling
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(layers.BatchNormalization())

#2nd Convolutional Layer
model.add(layers.Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid', activation='relu'))
#Max Pooling
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(layers.BatchNormalization())

#3rd Convolutional Layer
model.add(layers.Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
model.add(layers.BatchNormalization())

#Max Pooling
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 31, 31, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 256)       2973952   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 96)          2

In the next step we are going to compile the model. We are using the Adam optimizer. The loss function is "Sparse Categorical Crossentropy" as we predict only integers.

In [0]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [0]:
#fitting the model with our train data
model_trained = model.fit(X_train,y_train,epochs=20,validation_split=0.30, verbose=0)

In [0]:
#extracting test data
X_test, y_test = extract_imgs('/content/drive/My Drive/ImageClassification/seg_test/',100)

In [0]:
#evaluating the model with the test data
model.evaluate(X_test, y_test, verbose=2)

19/19 - 0s - loss: 1.0790 - accuracy: 0.7733


[1.0789562463760376, 0.7733333110809326]

As we can see our model has an accuracy of 77,3% on our test data.